In [ ]:
import sys; sys.path.extend([".."])

import json
import os

import cv2
import numpy as np
import matplotlib.colors
import matplotlib.pyplot as plt

from utils import resize_image, get_image_filtered_by_color, get_circle_centers

In [ ]:
colors_filedir = '../config.json'
image_filedir = '../data/output.png'
drawing_radius = 30
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
thickness = 2

In [ ]:
original = cv2.imread(image_filedir)
original = resize_image(original, 0.5)

drawing = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)[:, :, None]
drawing = np.repeat(drawing, 3, axis=-1)
drawing = cv2.convertScaleAbs(drawing, alpha=5, beta=100)

with open(colors_filedir) as f:
  datas = json.load(f)

image_filename = os.path.splitext(os.path.split(image_filedir)[1])[0]
save_path = os.path.join('../save', image_filename)

for name in datas.keys():
    hex_color = datas[name]['hex_color']
    os.makedirs(f'{save_path}/{name}', exist_ok=True)
    bgr_color = tuple([int(c * 255) for c in matplotlib.colors.to_rgb(hex_color)])[::-1]
    new_image = get_image_filtered_by_color(original, bgr_color)
    filedir = f'{save_path}/{name}/new_image.png'
    print(filedir)
    cv2.imwrite(filedir, new_image)
    centers = get_circle_centers(new_image)
    datas[name]['nof_circles'] = len(centers)
    for k, center in enumerate(centers):
        cv2.circle(drawing, center, drawing_radius, bgr_color, -1)
        cv2.putText(drawing, f'{k}', center, font, font_scale, (0, 0, 0), thickness, cv2.LINE_AA)

# sort
names = [name for name in datas.keys()]
values = [datas[name]['nof_circles'] for name in datas.keys()]
idxs = np.argsort(values)[::-1]
datas = {names[idx]: datas[names[idx]] for idx in idxs}

filedir = f'{save_path}/drawing.png'
print(filedir)
cv2.imwrite(filedir, drawing)

total_nof_circles = sum([datas[name]['nof_circles'] for name in datas.keys()])
print(f'total_nof_circles={total_nof_circles}')
for name in datas.keys():
    hex_color = datas[name]['hex_color']
    nof_circles = datas[name]['nof_circles']
    print(f'\t{name} hex_color={hex_color}; nof_circles={nof_circles} ({nof_circles/total_nof_circles*100:.3f}%)')


fig, ax = plt.subplots(1, 1, figsize=(20, 20))
ax.imshow(cv2.cvtColor(drawing, cv2.COLOR_BGR2RGB))
ax.axis('off')
plt.show()